In [2]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision.models as models
from torchvision import datasets, transforms
# For utilities
import os, shutil, time

In [3]:
use_gpu = torch.cuda.is_available()
use_gpu

True

In [4]:
class ColorizationNet(nn.Module):
  def __init__(self, input_size=128):
    super(ColorizationNet, self).__init__()
    MIDLEVEL_FEATURE_SIZE = 128

    ## First half: ResNet
    resnet = models.resnet18(num_classes=365) 
    # Change first conv layer to accept single-channel (grayscale) input
    resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    # Extract midlevel features from ResNet-gray
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])

    ## Second half: Upsampling
    self.upsample = nn.Sequential(     
      nn.Conv2d(MIDLEVEL_FEATURE_SIZE, 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1),
      nn.Upsample(scale_factor=2)
    )

  def forward(self, input):

    # Pass input through ResNet-gray to extract features
    midlevel_features = self.midlevel_resnet(input)

    # Upsample to get colors
    output = self.upsample(midlevel_features)
    return output

In [5]:
model = ColorizationNet()

In [6]:
criterion = nn.MSELoss()

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0)

In [8]:
class GrayscaleImageFolder(datasets.ImageFolder):
  '''Custom images folder, which converts images to grayscale before loading'''
  def __getitem__(self, index):
    path, target = self.imgs[index]
    img = self.loader(path)
    if self.transform is not None:
      img_original = self.transform(img)
      img_original = np.asarray(img_original)
      img_lab = rgb2lab(img_original)
      img_lab = (img_lab + 128) / 255
      img_ab = img_lab[:, :, 1:3]
      img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
      img_original = rgb2gray(img_original)
      img_original = torch.from_numpy(img_original).unsqueeze(0).float()
    if self.target_transform is not None:
      target = self.target_transform(target)
    return img_original, img_ab, target

In [9]:
# Training
train_transforms = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.RandomHorizontalFlip()])
train_imagefolder = GrayscaleImageFolder('dataset/train', train_transforms)
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=32, shuffle=True)

# Validation 
val_transforms = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224)])
val_imagefolder = GrayscaleImageFolder('dataset/validation' , val_transforms)
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=10, shuffle=False)

In [10]:
class AverageMeter(object):
  '''A handy class from the PyTorch ImageNet tutorial''' 
  def __init__(self):
    self.reset()
  def reset(self):
    self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
  '''Show/save rgb image from grayscale and ab channels
     Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
  plt.clf() # clear matplotlib 
  color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  if save_path is not None and save_name is not None: 
    plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [11]:
def validate(val_loader, model, criterion, save_images, epoch):
  model.eval()

  # Prepare value counters and timers
  batch_time, data_time, losses = AverageMeter(), AverageMeter(), AverageMeter()

  end = time.time()
  already_saved_images = False
  for i, (input_gray, input_ab, target) in enumerate(val_loader):
    data_time.update(time.time() - end)

    # Use GPU
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()

    # Run model and record loss
    output_ab = model(input_gray) # throw away class predictions
    loss = criterion(output_ab, input_ab)
    losses.update(loss.item(), input_gray.size(0))

    # Save images to file
    if save_images and not already_saved_images:
      already_saved_images = True
      for j in range(min(len(output_ab), 10)): # save at most 5 images
        save_path = {'grayscale': 'outputs/gray/', 'colorized': 'outputs/color/'}
        save_name = 'img-{}-epoch-{}.jpg'.format(i * val_loader.batch_size + j, epoch)
        to_rgb(input_gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    # Record time to do forward passes and save images
    batch_time.update(time.time() - end)
    end = time.time()

    # Print model accuracy -- in the code below, val refers to both value and validation
    if i % 25 == 0:
      print('Validate: [{0}/{1}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
             i, len(val_loader), batch_time=batch_time, loss=losses))

  print('Finished validation.')
  return losses.avg

In [12]:
def train(train_loader, model, criterion, optimizer, epoch):
  print('Starting training epoch {}'.format(epoch))
  model.train()
  
  # Prepare value counters and timers
  batch_time, data_time, losses = AverageMeter(), AverageMeter(), AverageMeter()

  end = time.time()
  for i, (input_gray, input_ab, target) in enumerate(train_loader):
    
    # Use GPU if available
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()

    # Record time to load data (above)
    data_time.update(time.time() - end)

    # Run forward pass
    output_ab = model(input_gray) 
    loss = criterion(output_ab, input_ab) 
    losses.update(loss.item(), input_gray.size(0))

    # Compute gradient and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Record time to do forward and backward passes
    batch_time.update(time.time() - end)
    end = time.time()

    # Print model accuracy -- in the code below, val refers to value, not validation
    if i % 25 == 0:
      print('Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
              epoch, i, len(train_loader), batch_time=batch_time,
             data_time=data_time, loss=losses)) 

  print('Finished training epoch {}'.format(epoch))

In [13]:
# Move model and loss function to GPU
if use_gpu: 
  criterion = criterion.cuda()
  model = model.cuda()

In [14]:
# Make folders and set parameters
os.makedirs('outputs/color', exist_ok=True)
os.makedirs('outputs/gray', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)
save_images = True
best_losses = 1e10
epochs = 1000

In [28]:
# Train model
for epoch in range(epochs):
  # Train for one epoch, then validate
  train(train_loader, model, criterion, optimizer, epoch)
  with torch.no_grad():
    losses = validate(val_loader, model, criterion, save_images, epoch)
  # Save checkpoint and replace old best model if current model is better
  if losses < best_losses:
    best_losses = losses
    torch.save(model.state_dict(), 'checkpoints/model-epoch-{}-losses-{:.3f}.pth'.format(epoch+1,losses))

Starting training epoch 0
Epoch: [0][0/5]	Time 5.567 (5.567)	Data 1.236 (1.236)	Loss 0.5903 (0.5903)	
Finished training epoch 0


E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 4 pixels
  return func(*args, **kwargs)


Validate: [0/4]	Time 0.986 (0.986)	Loss 14729264.0000 (14729264.0000)	
Finished validation.
Starting training epoch 1
Epoch: [1][0/5]	Time 1.131 (1.131)	Data 1.072 (1.072)	Loss 0.1958 (0.1958)	
Finished training epoch 1


E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 49600 pixels
  return func(*args, **kwargs)
E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 49875 pixels
  return func(*args, **kwargs)
E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 50156 pixels
  return func(*args, **kwargs)
E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 50140 pixels
  return func(*args, **kwargs)
E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 50176 pixels
  return func(*args, **kwargs)
E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 49339 pixels
  return func(*args, **kwargs)
E:\Softwares\Anaconda\lib\site-pac

Validate: [0/4]	Time 0.716 (0.716)	Loss 190188.5938 (190188.5938)	
Finished validation.
Starting training epoch 2
Epoch: [2][0/5]	Time 1.038 (1.038)	Data 0.991 (0.991)	Loss 0.0329 (0.0329)	
Finished training epoch 2


...

E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 81 pixels
  return func(*args, **kwargs)
E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 521 pixels
  return func(*args, **kwargs)
E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 560 pixels
  return func(*args, **kwargs)


Validate: [0/4]	Time 1.142 (1.142)	Loss 0.0085 (0.0085)	
Finished validation.
Starting training epoch 861
Epoch: [861][0/5]	Time 1.765 (1.765)	Data 1.703 (1.703)	Loss 0.0017 (0.0017)	
Finished training epoch 861
Validate: [0/4]	Time 1.124 (1.124)	Loss 0.0026 (0.0026)	
Finished validation.
Starting training epoch 862
Epoch: [862][0/5]	Time 2.407 (2.407)	Data 2.313 (2.313)	Loss 0.0010 (0.0010)	
Finished training epoch 862
Validate: [0/4]	Time 1.101 (1.101)	Loss 0.0025 (0.0025)	
Finished validation.
Starting training epoch 863
Epoch: [863][0/5]	Time 1.590 (1.590)	Data 1.527 (1.527)	Loss 0.0011 (0.0011)	
Finished training epoch 863
Validate: [0/4]	Time 1.141 (1.141)	Loss 0.0026 (0.0026)	
Finished validation.
Starting training epoch 864
Epoch: [864][0/5]	Time 1.727 (1.727)	Data 1.658 (1.658)	Loss 0.0011 (0.0011)	
Finished training epoch 864
Validate: [0/4]	Time 1.214 (1.214)	Loss 0.0025 (0.0025)	
Finished validation.
Starting training epoch 865
Epoch: [865][0/5]	Time 1.824 (1.824)	Data 1.75

E:\Softwares\Anaconda\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 38 pixels
  return func(*args, **kwargs)


Validate: [0/4]	Time 0.813 (0.813)	Loss 0.0041 (0.0041)	
Finished validation.
Starting training epoch 937
Epoch: [937][0/5]	Time 2.016 (2.016)	Data 1.954 (1.954)	Loss 0.0009 (0.0009)	
Finished training epoch 937
Validate: [0/4]	Time 2.141 (2.141)	Loss 0.0026 (0.0026)	
Finished validation.
Starting training epoch 938
Epoch: [938][0/5]	Time 2.141 (2.141)	Data 2.079 (2.079)	Loss 0.0008 (0.0008)	
Finished training epoch 938
Validate: [0/4]	Time 0.922 (0.922)	Loss 0.0025 (0.0025)	
Finished validation.
Starting training epoch 939
Epoch: [939][0/5]	Time 2.126 (2.126)	Data 2.032 (2.032)	Loss 0.0008 (0.0008)	
Finished training epoch 939
Validate: [0/4]	Time 2.704 (2.704)	Loss 0.0021 (0.0021)	
Finished validation.
Starting training epoch 940
Epoch: [940][0/5]	Time 2.891 (2.891)	Data 2.829 (2.829)	Loss 0.0009 (0.0009)	
Finished training epoch 940
Validate: [0/4]	Time 0.828 (0.828)	Loss 0.0026 (0.0026)	
Finished validation.
Starting training epoch 941
Epoch: [941][0/5]	Time 2.814 (2.814)	Data 2.75

<Figure size 640x480 with 0 Axes>